# Singular Value Decomposition (CF)

CF в любом из вариантов (**user-based** / **item-based**) имеет проблемы, если матрица сильно разрежена, а также имеет проблему масштабируемости. Это не позволяет использовать решения на основе CF на очень больших данных. 

Описание проблемы разреженности  [Sarwar et al. (2000)](http://files.grouplens.org/papers/webKDD00.pdf), где предлагается *Singular Value Decomposition (SVD)* для решения.

---

### Как?

SVD разкладывает матрицу размера $m\times n$ в матрицы $P$, $\Sigma$ и $Q$:

\begin{equation}
R = P\Sigma Q^{\top}.
\end{equation}

$P$ и $Q$ это ортогональные матрицы и $\Sigma$ диагональная матрица состоящая из сингулярных значений рейтингов в качестве диагональных значений ([Billsus and Pazzani, 1998](https://www.ics.uci.edu/~pazzani/Publications/MLC98.pdf), [Sarwar et al. (2000)](http://files.grouplens.org/papers/webKDD00.pdf)).

![](img/svd.png)

Матрица рейтингов рассчитывается: 

\begin{equation}
R_k = P_k\Sigma_k Q_k^{\top}.
\end{equation}


---

### SVD 

> 1. Нормализованная матрица $R_{norm}$ раскладывается на $P$, $\Sigma$ и $Q$
> 2. Уменьшаем $\Sigma$ до размерности $k$ и трансформируем в $\Sigma_k$
> 3. Считаем квадратный корень из $\Sigma_k$ для получения $\Sigma_k^{\frac{1}{2}}$
> 4. Считаем финальную матрицу $P_k\Sigma_k^{\frac{1}{2}}$ и $\Sigma_k^{\frac{1}{2}}Q_k^{\top}$, которая будет использоваться для расчета рекомендаций.

---


In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np
import zipfile

import os
import sys

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [ ]:
pd.crosstab(ratings.userId, ratings.movieId, ratings.rating, aggfunc=sum)

In [ ]:
def rating_matrix(ratings):
    """
    1. Запоним NaN средним рейтингом
    2. Нормализуем рейтинг относительно среднего значения
    
    :param ratings : DataFrame
    :return
        - R : Numpy array рейтингов
        - df : DataFrame рейтингов
    """
    
    # средний рейтинг
    umean = # ваш код
    
    # создаем матрицу и заполняем средний с параметром axis =0 пустые значения
    df = # ваш код
    df = # ваш код
    
    # нормализация по среднему значению
    df = # ваш код
    
    # в numpy
    R = df.to_numpy()
    
    return R, df

# записываем результат
R, df = rating_matrix(ratings)

$R$ готовая матрица для применения

In [ ]:
df

### SVD


1. ```fit()``` - расчет SVD рейтингов и сохранение матриц P, S, Q
2. ```predict()``` - матрицы P, S и Qh для создания предикта по пользователю - элементу. Учитывая, что мы сделали вычитание фактического рейтинга из среднего по пользователю, нам необходимо будет вернуть значение, прибавив предикт к среднему рейтингу
3. ```recommend()``` - функция рекомендаций

In [ ]:
class SVD:
    
    def __init__(self, umeam):
        """
        :param
            - umean : среднее значение рейтингов по пользователю
        """
        self.umean = umean.to_numpy()
        
        # init svd 
        self.P = np.array([])
        self.S = np.array([])
        self.Qh = np.array([])
        
        # init пользователь и элемент
        self.u_factors = np.array([])
        self.i_factors = np.array([])
    
    def fit(self, R):
        """
        Fit  SVD
        """
        P, s, Qh = np.linalg.svd(R, full_matrices=False)
        
        self.P = P
        self.S = np.diag(s)
        self.Qh = Qh
        
        # скрытые факторы по пользователю (u_factors) и по элементу (i_factors)
        self.u_factors = # ваш код факторов по пользователю 
        self.i_factors = # ваш код факторов по элементу 
    
    def predict(self, userid, itemid):
        """
        Предикт по пользователю
        
        :param
            - userid : пользователь
            - itemid : элемент
            
        :return
            - r_hat : predicted rating
        """
        
        # предикт вычисляется по факторам пользователя и элемента
        r_hat = # примените функцию из numpy для получения суммы по всем факторам (векторам)
        
        # суммируем со средним значением
        r_hat += self.umean[userid]
        
        return r_hat
        
    
    def recommend(self, userid):
        """
        :param
            - userid : id пользователя
        """
        
        # предикт для пользователя по факторам     
        predictions = # примените функцию из numpy для получения суммы по всем факторам (векторам), не забудьте прибавить среднее значение
        
        # сортировка результата
        top_idx = np.flip(np.argsort(predictions))
        preds = predictions[top_idx]
        
        return top_idx, preds
        

Создадим SVD модель

Передадим средний рейтинг, как базовый элемент

In [ ]:
umean = ratings.groupby(by='userId')['rating'].mean()

# svd
svd = SVD(umean)

# fit
svd.fit(R)

### Предикт рейтинга

In [ ]:
ratings.head(10)

In [ ]:
# для кого делаем предикт
userid = 1

# какие элементы подбираем
items = [1,3,6,47,50,70,101,110,151,157]

# формирование предикта
for itemid in items:
    r = svd.predict(userid=userid, itemid=itemid)
    print('предикт для пользователя ={} по элементу ={} : {}'.format(userid, itemid, r))

### Создание рекомендаций

In [ ]:
userid = 1

# сортировка предикта
top_indx, preds = svd.recommend(userid=userid)

rec_movies = movies[movies['movieId'].isin(top_indx)]

# списко элементов, которые пользователь отметил рейтингом
uitems = ratings.loc[ratings.userId == userid].movieId.to_list()

# убираем фильмы уже оцененные пользователем
top10 = np.setdiff1d(top_indx, uitems, assume_unique=True)[:10]

# топ - N
top10_idx = list(np.where(top_indx == idx)[0][0] for idx in top10)
top10_predictions = preds[top10_idx]

# добавляем название и жанр
zipped_top10 = list(zip(top10,top10_predictions))
top10 = pd.DataFrame(zipped_top10, columns=['movieId','predictions'])
List = pd.merge(top10, movies, on='movieId', how='inner')

List